# Rpy2 Test
Try this package out, it seems to be the most popular. Instead of calling entire R scripts from the python code this provides a pythonic way to access R packages which will make development easier

In [75]:
%reset -f

In [76]:
import rpy2
import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages
from rpy2.robjects.vectors import StrVector

from rpy2.robjects import r, pandas2ri
pandas2ri.activate()

Note that in python script we will want to use a path based on file code was run from using something more like 
`dir_path = os.path.dirname(os.path.realpath(__file__))`

In [77]:
import os
cwd = os.getcwd()
data_dir = cwd.rsplit('/', 1)[0] +'/data'

In [78]:
f'read.csv("{data_dir}/HVS_data.csv", header=True)'

'read.csv("/Users/alexanderweinstein/Documents/Data_Products/db-equitable-development-tool/data/HVS_data.csv", header=True)'

In [79]:

data = robjects.r(f'read.csv("{data_dir}/HVS_data.csv", header=TRUE)')
data.head()

,recid,boro,uf1_1,uf1_3,uf1_4,uf1_5,uf1_6,uf1_7,uf1_8,uf1_9,...,fw74,fw75,fw76,fw77,fw78,fw79,fw80,il30per,il50per,il80per
1,1,1,9,9,9,1,9,9,9,9,...,37635401,33263895,10348898,11446240,32161257,9745805,13739972,21800,36250,58000
2,1,1,9,9,9,1,9,9,9,9,...,8270628,23614216,42797335,19594333,8525261,36330659,28940264,27200,45300,72500
3,1,1,9,9,9,1,9,9,9,9,...,8270628,23614216,42797335,19594333,8525261,36330659,28940264,24500,40800,65250
4,1,1,9,9,9,1,9,9,9,9,...,35934838,10027288,46175949,57167298,43401081,10664017,66729558,21800,36250,58000
5,1,1,9,9,9,1,9,9,9,9,...,50290497,10630692,32310018,70148673,42706530,10137630,64932695,21800,36250,58000


In [80]:
type(data)

pandas.core.frame.DataFrame

Ok sick could load in pandas from .pkl to see if they match?  Save that for later I think

For now do data cleaning in python instead of R

In [81]:
weights_correction = 10**5
data['fw'] = data['fw']/weights_correction

In [82]:
rep_weights_columns = [f'fw{i}' for i in range(1,81)]
data[rep_weights_columns] = data[rep_weights_columns]/weights_correction

Add occupancy final column

In [83]:
data['occ_final'] = data['recid']==1

Use survey package

In [84]:
survey_package = rpackages.importr('survey')
base = rpackages.importr('base')

In [ ]:
survey_package = rpackages.importr('survey')
hu_design = survey_package.svrepdesign(variables=data.iloc[:, :188],
                          repweights=data[rep_weights_columns],
                          weights=data['fw'], combined_weights = True, type='other',
                         scale=4/80, rscales=1)

In [103]:
hu_design[4]

Rlang( svrepdesign.default(combined_weights = TRUE, type = "other",  )

In [86]:
occ_subset = base.subset(hu_design, data['occ_final']==1)
survey_total = survey_package.svytotal(data['occ_final'], design=occ_subset)


In [87]:
survey_total

array([3109954.62348])

Ok correct total but no standard error. Close but not there yet. What happens when we group by sub borough area?

In [88]:
by_sub_borough_area = survey_package.svyby(x=data['occ_final'], by=data[['boro','cd']], 
                                           design=hu_design, FUN=survey_package.svytotal)
by_sub_borough_area

,boro,cd,V1,se
1.1,1,1,48974.64704,2261.013620
2.1,2,1,63928.61033,2963.900921
3.1,3,1,81269.27401,3000.429807
4.1,4,1,74744.73583,2710.182060
5.1,5,1,59787.02995,2994.213641
1.2,1,2,54465.07901,2894.219802
2.2,2,2,56126.65940,2836.227055
3.2,3,2,74125.68599,3313.716272
4.2,4,2,50870.50410,2296.777069
5.2,5,2,46583.72836,2204.576701


Ok same measurements of total housing units as R but different standard errors. 

In [111]:
data['fw'][:10]

1     340.40517
2     245.77110
3     245.77110
4     370.98358
5     370.41707
6     203.19779
7     242.22272
8     317.84919
9     223.39117
10    247.05536
Name: fw, dtype: float64